## Import Semua Packages/Library yang Digunakan

In [ ]:
# Library yang sering digunakan
import os, shutil
import zipfile
import random
from random import sample
import shutil
from shutil import copyfile
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm as tq # untuk menampilkna bar progres saat proses iterasi

# Libraries untuk pemrosesan data gambar
import cv2
from PIL import Image
import skimage
from skimage import io
from skimage.transform import resize
from skimage.transform import rotate, AffineTransform, warp
from skimage import img_as_ubyte
from skimage.exposure import adjust_gamma
from skimage.util import random_noise

# Libraries untuk pembangunan model
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras import Model, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import InputLayer, Conv2D, SeparableConv2D, MaxPooling2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau

# Import the required library
import os
import random
import shutil
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from PIL import Image, ImageDraw, ImageOps
import seaborn as sns
from google.colab import files

In [ ]:
# Untuk menonaktifkan warinig yang mungkin muncul, seperti FutureWarning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Mencetak versi TensorFlow yang sedang digunakan
print(tf.__version__)

In [ ]:
# Menggunakan Google Colab
!nvidia-smi

## Data Preparation

### Data Loading

In [ ]:
# Import module yang disediakan google colab untuk kebutuhan upload file

from google.colab import files
files.upload()

In [ ]:
# Buat direktori jika belum ada

os.makedirs("/root/.kaggle", exist_ok=True)

# Pindahkan file ke direktori .kaggle

shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

# Atur permission agar tidak terlalu terbuka

os.chmod("/root/.kaggle/kaggle.json", 600)


In [ ]:
!kaggle datasets download cookiefinder/tomato-disease-multiple-sources

In [ ]:
!unzip tomato-disease-multiple-sources.zip

### Dataset Checking


In [ ]:
import os
import shutil

In [ ]:
# Direktori awal untuk train dan test
train_dir = 'train'
test_dir = 'valid'

# Direktori baru untuk dataset gabungan
combined_dir = "Tomato-Disease/dataset"

In [ ]:
# direktori baru untuk dataset gabungan
os.makedirs(combined_dir, exist_ok=True)

In [ ]:
# Daftar kelas penyakit daun non-parasit
leaf_disease_class = [
    "Bacterial_spot",
    "Early_blight",
    "Late_blight",
    "Leaf_Mold",
    "Septoria_leaf_spot",
    "Target_Spot",
    "healthy",
    "powdery_mildew"
]

# Salin folder dari train_dir
for category in os.listdir(train_dir):
    if category in leaf_disease_class:
        category_dir = os.path.join(train_dir, category)
        if os.path.isdir(category_dir):
            shutil.copytree(category_dir, os.path.join(combined_dir, category), dirs_exist_ok=True)

# Salin folder dari test_dir
for category in os.listdir(test_dir):
    if category in leaf_disease_class:
        category_dir = os.path.join(test_dir, category)
        if os.path.isdir(category_dir):
            shutil.copytree(category_dir, os.path.join(combined_dir, category), dirs_exist_ok=True)

In [ ]:
path = 'Tomato-Disease/dataset'
rice_image = {}

# Hanya ambil folder (class) yang valid
for i in os.listdir(path):
    class_path = os.path.join(path, i)
    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        rice_image[i] = images

# menampilkan secara acak 5 gambar di bawah setiap kelas dari data
path_sub = "Tomato-Disease/dataset"

# menampilkan secarak acak 5 gambar di bawah setiap kelas dari data latih
fig, axs = plt.subplots(len(rice_image.keys()), 5, figsize=(15, 15))

for i, class_name in enumerate(rice_image.keys()):
    images = rice_image[class_name]

    if len(images) >= 5:
        sampled_images = np.random.choice(images, 5, replace=False)
    else:
        print(f"[WARNING] Kelas '{class_name}' hanya memiliki {len(images)} gambar. Menampilkan semua.")
        sampled_images = images + [''] * (5 - len(images))  # padding kosong agar tidak error

    for j, image_name in enumerate(sampled_images):
        ax = axs[i, j]
        if image_name != '':
            img_path = os.path.join(path, class_name, image_name)
            img = Image.open(img_path)
            ax.imshow(img)
        ax.set(xlabel=class_name, xticks=[], yticks=[])

fig.tight_layout()

### Plot Distribusi

In [ ]:
# path sumber
tomato_path = 'Tomato-Disease/dataset'

# daftar untuk menyimpan data untuk setiap nama file, path file, dan label dalam data
file_name = []
labels = []
full_path=[]

# mengambil file gamber, path file, dan label
for path, subdirs, files in os.walk(tomato_path):
  for name in files:
    full_path.append(os.path.join(path, name))
    labels.append(path.split('/')[-1])
    file_name.append(name)

distribution_train = pd.DataFrame({"path":full_path, 'file_name':file_name, 'labels':labels})

# plot distribusi
Label = distribution_train['labels']
plt.figure(figsize = (6, 6))
sns.set_style("darkgrid")
plot_data = sns.countplot(Label)

### Data Preprocessing

#### Data Augmentation

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import numpy as np
# import random
# from skimage.transform import resize
# from skimage.filters import gaussian
# from skimage import img_as_float

# # Membuat fungsi untuk melakukan rotasi berlawanan arah jarum jam
# def anticlockwise_rotation(img):
#     img = tf.image.resize(img, (224, 224))
#     img = tf.image.rot90(img, k=random.randint(1, 4))  # Rotasi 90, 180, atau 270 derajat secara acak
#     return img

# # Membuat fungsi untuk melakukan rotasi searah jarum jam
# def clockwise_rotation(img):
#     img = tf.image.resize(img, (224, 224))
#     img = tf.image.rot90(img, k=random.randint(1, 4))  # Rotasi 90, 180, atau 270 derajat secara acak
#     return img

# # Membuat fungsi untuk membalik gambar secara vertikal dari atas ke bawah
# def flip_up_down(img):
#     img = tf.image.resize(img, (224, 224))
#     img = tf.image.flip_up_down(img)
#     return img

# # Membuat fungsi untuk memberikan efek peningkatan kecerahan pada gambar
# def add_brightness(img):
#     img = tf.image.resize(img, (224, 224))
#     img = tf.image.adjust_brightness(img, delta=random.uniform(0.1, 0.5))  # Sesuaikan nilai delta sesuai kebutuhan
#     return img

# # Membuat fungsi untuk memberikan efek blur pada gambar
# def blur_image(img):
#     img = resize(img, (224, 224), preserve_range=True)
#     img = img_as_float(img)  # pastikan nilainya antara 0 dan 1
#     img = gaussian(img, sigma=1, channel_axis=-1)  # jika RGB, pastikan channel_axis benar
#     return img

# # Membuat fungsi untuk memberikan efek pergeseran acak pada gambar
# def sheared(img):
#     img = tf.image.resize(img, (224, 224))
#     # Buat objek ImageDataGenerator dengan parameter shearing range
#     datagen = ImageDataGenerator(shear_range=0.2)
#     img = next(iter(datagen.flow(tf.expand_dims(img, 0))))[0]
#     return img

# # Membuat fungsi untuk melakukan pergeseran melengkung pada gambar
# def warp_shift(img):
#     img = tf.image.resize(img, (224, 224))
#     # Buat objek ImageDataGenerator dengan parameter width_shift_range dan height_shift_range
#     datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1)
#     img = next(iter(datagen.flow(tf.expand_dims(img, 0))))[0]
#     return img

In [ ]:
# transformations = {
#     'rotate anticlockwise': anticlockwise_rotation,
#     'rotate clockwise': clockwise_rotation,
#     'warp shift': warp_shift,
#     'blurring image': blur_image,
#     'add brightness': add_brightness,
#     'flip up down': flip_up_down,
#     'shear image': sheared
# }

# images_path = "tomato_disease/dataset/powdery_mildew"  # Path for the original image
# augmented_path = "tomato_disease/dataset/powdery_mildew_augmented"  # Path to put the augmented image
# images = []  # To save images that have been preprocessed from the folder

# # Baca nama gambar dari folder dan tambahkan path ke dalam array "images"
# for im in os.listdir(images_path):
#     images.append(os.path.join(images_path, im))

# # Jumlah gambar yang akan ditambahkan dengan hasil transformasi augmentasi
# images_to_generate = 2000
# i = 1

# while i <= images_to_generate:
#     image = random.choice(images)
#     try:
#         original_image = io.imread(image)

#         # Memeriksa apakah gambar memiliki dimensi yang valid (3 atau 4 dimensi)
#         if original_image.ndim not in [3, 4]:
#             raise ValueError('Invalid image dimensions')

#         transformed_image = None
#         n = 0  # Variabel untuk melakukan iterasi sampai jumlah transformasi yang akan diterapkan
#         transformation_count = random.randint(1, len(transformations))  # Pilih jumlah transformasi acak yang akan diterapkan pada gambar

#         while n <= transformation_count:
#             key = random.choice(list(transformations))  # Secara acak memilih dan memanggil metode
#             transformed_image = transformations[key](original_image)
#             n = n + 1

#         new_image_path = "%s/augmented_image_%s.jpg" % (augmented_path, i)
#         transformed_image = img_as_ubyte(transformed_image)  # Mengonversi gambar ke format byte yang tidak ditandatangani, dengan nilai dalam [0, 255]
#         cv2.imwrite(new_image_path, transformed_image)  # Simpan hasil transformasi augmentasi pada gambar ke path yang ditentukan
#         i = i + 1
#     except ValueError as e:
#         print('Could not read or process the image', image, ':', e, 'hence skipping it.')

In [ ]:
# # Define source path
# tomato_path = "tomato_disease/dataset/"

# # Create a list that stores data for each filenames, filepaths, and labels in the data
# file_name = []
# labels = []
# full_path = []

# # Get data image filenames, filepaths, labels one by one with looping, and store them as dataframe
# for path, subdirs, files in os.walk(tomato_path):
#     for name in files:
#         full_path.append(os.path.join(path, name))
#         labels.append(path.split('/')[-1])
#         file_name.append(name)

# distribution_train = pd.DataFrame({"path":full_path,'file_name':file_name,"labels":labels})

# # Plot the distribution of images across the classes
# Label = distribution_train['labels']
# plt.figure(figsize = (6,6))
# sns.set_style("darkgrid")
# plot_data = sns.countplot(Label)

#### Split Dataset

In [ ]:
mypath = 'Tomato-Disease'

file_name = []
labels = []
full_path = []
for path, subdirs, files in os.walk(mypath):
  for name in files:
    full_path.append(os.path.join(path, name))
    labels.append(path.split('/')[-1])
    file_name.append(name)

df = pd.DataFrame({'path':full_path, 'file_name':file_name, 'labels':labels})
df.groupby(['labels']).size()

In [ ]:
X = df['path']
y = df['labels']

# Step 1: Split jadi train (80%) dan temp (val+test 20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=300, stratify=y)

# Step 2: Split temp jadi val dan test, masing-masing 50% dari temp (10% dari total awal)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=300, stratify=y_temp)

In [ ]:
# menyatukan ke dalam masing masing dataframe
df_train = pd.DataFrame({'path':X_train, 'labels':y_train,'set':'train'})
df_test = pd.DataFrame({'path':X_test, 'labels':y_test,'set':'test'})
df_val = pd.DataFrame({'path':X_val, 'labels':y_val,'set':'val'})

In [ ]:
# Print hasil diatas untuk melihat panjang size data training dan testing
print('train_size',len(df_train))
print('test_size',len(df_test))
print('val_size',len(df_val))


In [ ]:
# Gabungkan DataFrame df_tr dan df_te
df_all = pd.concat([df_train, df_test, df_val], ignore_index=True)

print('===================================================== \n')
print(df_all.groupby(['set', 'labels']).size(), '\n')
print('===================================================== \n')

# Cek sample data
print(df_all.sample(5))

In [ ]:
# memanggil dataset asli
datasource_path = 'Tomato-Disease/dataset/'
# membuat variabel dataset, tampat menampung data yng telah dilakukan split
dataset_path = 'Dataset-Final/'

In [ ]:
for index, row in tq(df_all.iterrows()):
  # Deteksi filepath
  file_path = row['path']
  if os.path.exists(file_path) == False:
    file_path = os.path.join(datasource_path, row['labels'], row['iamge'].split('.')[0])

  # buat direktori tujuan folder
  if os.path.exists(os.path.join(dataset_path, row['set'], row['labels'])) == False:
    os.makedirs(os.path.join(dataset_path, row['set'], row['labels']))

  # tentukan tujuan file
  destination_file_name = file_path.split('/')[-1]
  file_dest = os.path.join(dataset_path, row['set'], row['labels'], destination_file_name)

  # salin file dari sumber ke tujuan
  if os.path.exists(file_dest) == False:
    shutil.copy2(file_path, file_dest)

## Image Data Generator

In [ ]:
# definisikan direktori
TRAIN_DIR = 'Dataset-Final/train/'
TEST_DIR = 'Dataset-Final/test/'
VAL_DIR = 'Dataset-Final/val/'

# Ambil semua nama kelas dari folder train
classes = os.listdir(TRAIN_DIR)

# Buat dictionary untuk menyimpan jumlah gambar per kelas
train_counts = {}
test_counts = {}
val_counts = {}

for cls in classes:
    train_path = os.path.join(TRAIN_DIR, cls)
    test_path = os.path.join(TEST_DIR, cls)
    val_path = os.path.join(VAL_DIR, cls)

    train_counts[cls] = len(os.listdir(train_path))
    test_counts[cls] = len(os.listdir(test_path))
    val_counts[cls] = len(os.listdir(val_path))


# Tampilkan jumlah gambar per kelas
print("Jumlah gambar pada dataset training:")
for cls, count in train_counts.items():
    print(f"{cls}: {count}")

print("\nJumlah gambar pada dataset testing:")
for cls, count in test_counts.items():
    print(f"{cls}: {count}")

print("\nJumlah gambar pada dataset validation:")
for cls, count in val_counts.items():
    print(f"{cls}: {count}")

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
# objek ImageDataGenerator yang menormalkan gambar
datagen = ImageDataGenerator(rescale=1/255.,
                             validation_split = 0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(TRAIN_DIR,
                                              batch_size=32,
                                              target_size=(224, 224),
                                              color_mode='grayscale',
                                              class_mode='categorical',
                                              shuffle=True)

validation_generator = datagen.flow_from_directory(VAL_DIR,
                                                   batch_size=32,
                                                   target_size=(224, 224),
                                                   color_mode='grayscale',
                                                   class_mode='categorical',
                                                   shuffle=False)

test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                  batch_size=1,
                                                  target_size=(224, 224),
                                                  color_mode='grayscale',
                                                  class_mode='categorical',
                                                  shuffle=False)

## Modelling

### Skema 1 - CNN Architecture

In [ ]:
model_1 = Sequential()

# 1st
model_1.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 1)))
model_1.add(BatchNormalization())
model_1.add(MaxPooling2D((2, 2)))

# 2nd
model_1.add(Conv2D(32, (4, 4), padding='same', activation='relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling2D((2, 2)))

# 3rd
model_1.add(Conv2D(32, (7, 7), padding='same', activation='relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling2D((2, 2)))

model_1.add(Flatten())
model_1.add(Dense(128, activation='relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(64, activation='relu'))
model_1.add(Dropout(0.3))
model_1.add(Dense(8, activation='softmax'))

# compiler
model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,), loss='categorical_crossentropy', metrics=['accuracy'])
print(model_1.summary())

In [ ]:
# menghitung weight tiap kelas untuk mengatasi data yang imbalance

# Ambil nama kelas
classes = os.listdir(TRAIN_DIR)

# Hitung jumlah gambar tiap kelas
class_counts = {cls: len(os.listdir(os.path.join(TRAIN_DIR, cls))) for cls in classes}

# Total semua data
total = sum(class_counts.values())

# Hitung class_weights
class_weights = {
    i: (1 / count) * (total / len(class_counts))
    for i, (cls, count) in enumerate(class_counts.items())
}

# Buat mapping label kelas ke index (yang sesuai dengan class_indices dari ImageDataGenerator)
class_indices = {cls: i for i, cls in enumerate(sorted(classes))}

print("Class indices:", class_indices)
print("Class weights:", class_weights)

In [ ]:
# callbacks
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Fitting / Training model
history_1 = model_1.fit(train_generator,
                        epochs=30,
                        batch_size=32,
                        validation_data=validation_generator,
                        class_weight= class_weights,
                        callbacks=[checkpoint, early_stopping])

## Skema 2 - MobileNet v2 Transfer Learning


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
# objek ImageDataGenerator yang menormalkan gambar
datagen = ImageDataGenerator(rescale=1/255.,
                             validation_split = 0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(TRAIN_DIR,
                                              batch_size=32,
                                              target_size=(224, 224),
                                              color_mode='rgb',
                                              class_mode='categorical',
                                              shuffle=True)

validation_generator = datagen.flow_from_directory(VAL_DIR,
                                                   batch_size=32,
                                                   target_size=(224, 224),
                                                   color_mode='rgb',
                                                   class_mode='categorical',
                                                   shuffle=False)

test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                  batch_size=1,
                                                  target_size=(224, 224),
                                                  color_mode='rgb',
                                                  class_mode='categorical',
                                                  shuffle=False)

In [ ]:
# Initialize pre-trained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze all layers

# Creating a new model
model_2 = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    layers.Dropout(0.4),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(8, activation='softmax')  # 8 class
])

model_2.summary()

In [ ]:
# Compile the model
model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training model_2
history = model_2.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    class_weight=class_weights,
    callbacks=[lr_scheduler, checkpoint, early_stopping],
    verbose=1
)

# Model_2 evaluation on validation data
val_loss, val_acc = model_2.evaluate(validation_generator)
print(f'Validation accuracy: {val_acc * 100:.2f}%')

### Fine Tuning

In [ ]:
# Unfreeze some of the last layers of the base model
base_model.trainable = True
for layer in base_model.layers[-30:]:
    layer.trainable = False

# Callbacks
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Compile the model
model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tuning training
history_fine_tune = model_2.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    class_weight=class_weights,
    callbacks=[lr_scheduler, checkpoint, early_stopping],
    verbose=1
)

# Model_2 evaluation on validation data
val_loss, val_acc = model_2.evaluate(validation_generator)
print(f'Validation accuracy: {val_acc * 100:.2f}%')

## Evaluasi dan Visualisasi

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Reset generator agar prediksi dimulai dari awal
test_generator.reset()

# prediksi model
preds_1 = model_2.predict(test_generator, verbose=0)

# Ambil prediksi kelas
predicted_classes = np.argmax(preds_1, axis=1)

# Label ground truth dari generator
true_classes = test_generator.classes

# Nama kelas (harus sesuai urutan class_indices dari generator)
class_labels = list(test_generator.class_indices.keys())

# Print Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)
cm_df = pd.DataFrame(cm, index=class_labels, columns=[f'Predicted {label}' for label in class_labels])

plt.figure(figsize=(10, 7))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Class')
plt.xlabel('Predicted Class')
plt.show()

# Print Classification Report
print("\nClassification Report:\n")
print(classification_report(true_classes, predicted_classes, target_names=class_labels, digits=4))

In [ ]:
acc = history_fine_tune.history['accuracy']
val_acc = history_fine_tune.history['val_accuracy']
loss = history_fine_tune.history['loss']
val_loss = history_fine_tune.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and Validation Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.title('Training and Validaion Loss')
plt.show()


## Konversi Model

### SaveModel

In [ ]:
save_path = 'mymodel/'
tf.saved_model.save(model_2, save_path)

###Format tf.js

In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    /content/mymodel/ \
    /content/modeltfjs

### Format tflite

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("mymodel")
tflite_model = converter.convert()

with tf.io.gfile.GFile('tomato_disease.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
model_2.save("tanamind_tomato.h5")

In [ ]:
from google.colab import files
files.download('tanamind_tomato.h5')

## Inference (Optional)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import random

# Ambil probabilitas prediksi dari model
probabilities = model_2.predict(test_generator, verbose=1)

# Ambil label kelas dari generator
class_labels = list(test_generator.class_indices.keys())

# Pilih 5 gambar acak dari dataset
random_indices = random.sample(range(len(test_generator.filenames)), 5)

# Atur layout gambar dalam 1 baris
plt.figure(figsize=(15, 3))

for i, index in enumerate(random_indices):
    probability = probabilities[index]
    image_path = os.path.join(TEST_DIR, test_generator.filenames[index])
    img = mpimg.imread(image_path)

    # Prediksi dan ambil label dengan probabilitas tertinggi
    predicted_class_index = np.argmax(probability)
    confidence = probability[predicted_class_index] * 100
    predicted_label = class_labels[predicted_class_index]

    # Tampilkan gambar dengan judul prediksi
    plt.subplot(1, 5, i+1)  # 1 baris, 5 kolom, gambar ke-i
    plt.imshow(img)
    plt.axis('off')  # Matikan axis
    plt.title(f"{predicted_label}\n{confidence:.2f}%")

plt.tight_layout()  # Mengatur layout agar gambar tidak bertumpuk
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# Ganti dengan path file gambar lokal
image_path = 'test.jpg'

# Load dan praproses gambar
img = image.load_img(image_path, target_size=(150, 150))  # sesuaikan ukuran input model
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # tambah dimensi batch
img_array = img_array / 255.0  # normalisasi jika diperlukan

# Lakukan prediksi
prediction = model_2.predict(img_array)

# Ambil label kelas
class_labels = list(test_generator.class_indices.keys())

# Dapatkan hasil prediksi
predicted_index = np.argmax(prediction[0])
predicted_label = class_labels[predicted_index]
confidence = prediction[0][predicted_index] * 100

# Tampilkan gambar dan hasil prediksi
plt.imshow(img)
plt.axis('off')
plt.title(f"{predicted_label}\n{confidence:.2f}%")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# Ganti dengan path file gambar lokal
image_path = 'sehat.jpg'

# Load dan praproses gambar
img = image.load_img(image_path, target_size=(150, 150))  # sesuaikan ukuran input model
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # tambah dimensi batch
img_array = img_array / 255.0  # normalisasi jika diperlukan

# Lakukan prediksi
prediction = model_2.predict(img_array)

# Ambil label kelas
class_labels = list(test_generator.class_indices.keys())

# Dapatkan hasil prediksi
predicted_index = np.argmax(prediction[0])
predicted_label = class_labels[predicted_index]
confidence = prediction[0][predicted_index] * 100

# Tampilkan gambar dan hasil prediksi
plt.imshow(img)
plt.axis('off')
plt.title(f"{predicted_label}\n{confidence:.2f}%")
plt.show()